In [ ]:
using LCIO
using StatsPlots
using LinearAlgebra: norm
using JLD2
using IterableTables
gr()

In [ ]:
fileList = filter(s->occursin(r"E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8\.pR0.3\.\d\d_DST.slcio", s),
    readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

In [ ]:
#strange quarks
#note - kaon and pion momentum as measured by the detector
Momcut = 1.87/2
costhetacut = 1657.0/2084.68

Momsspion = Float64[]
TOFsspion = Float64[]
Momsskaon = Float64[]
TOFsskaon = Float64[]
Momsspionmc = Float64[]
Momsskaonmc = Float64[]
function main()
    filenum = 0
    count=0
    for file in fileList
        println(file)
        numEvents = 0
        filenum+=1
        LCIO.open(file) do reader
        for (iEvent, event) in enumerate(reader)
            numEvents+=1
            rp = getCollection(event, "PandoraPFOs")
            mcp = getCollection(event, "MCParticle")
            recoLinks = getCollection(event, "RecoMCTruthLink")
            rel=LCIO.LCRelationNavigator(recoLinks)
            coll = getCollection(event, "ECalBarrelHits")
            coll2 = getCollection(event, "ECalEndcapHits")
            decode = CellIDDecoder(coll)
            cellinfo = Dict()
                for hit in coll
                    layer = decode(hit)["layer"]
                    if layer != 0
                        continue
                    end
                    for i=1:LCIO.getNMCContributions(hit)
                        mcparticle = LCIO.getParticleCont(hit, i)
                        cell = LCIO.getCellID0(hit)
                        if LCIO.getCharge(mcparticle)==0
                            continue
                        end
                        recoparticle = getRelatedFromObjects(rel, mcparticle)
                        #if length(getRelatedToObjects(rel, mcparticle))!= 1  continue end
                        if length(recoparticle) != 1 continue end
                        recoparticle = recoparticle[1]
                        mom = norm(getMomentum(recoparticle))
                        mom2 = norm(getMomentum(mcparticle))
                        TOF = LCIO.getTimeCont(hit,i)
                        Mom3 = getMomentum(mcparticle)
                        costheta = Mom3[3]/sqrt(Mom3[1]^2+Mom3[2]^2+Mom3[3]^2)
                        if i>1
                            if TOF<LCIO.getTimeCont(hit,i-1)
                                count+=1
                            end
                        end
                        if TOF>10 || mom>10 || mom2>10 || mom<Momcut || mom2<Momcut || costheta>costhetacut
                            continue
                        end
                        if !(haskey(cellinfo, cell))
                            push!(cellinfo, cell => (mcparticle, recoparticle, TOF))
                        else
                            if cellinfo[cell][3]>TOF
                                cellinfo[cell]=(mcparticle, recoparticle, TOF)
                            end
                        end
                        
                    end
                end
                for hit in coll2
                    layer = decode(hit)["layer"]
                    if layer != 0
                        continue
                    end
                    for i=1:LCIO.getNMCContributions(hit)
                        mcparticle = LCIO.getParticleCont(hit, i)
                        cell = LCIO.getCellID0(hit)
                        if LCIO.getCharge(mcparticle)==0
                            continue
                        end
                        recoparticle = getRelatedFromObjects(rel, mcparticle)
                        #if length(getRelatedToObjects(rel, mcparticle))!= 1  continue end
                        if length(recoparticle) != 1 continue end
                        recoparticle = recoparticle[1]
                        mom = norm(getMomentum(recoparticle))
                        mom2 = norm(getMomentum(mcparticle))
                        TOF = LCIO.getTimeCont(hit,i)
                        Mom3 = getMomentum(mcparticle)
                        costheta = Mom3[3]/sqrt(Mom3[1]^2+Mom3[2]^2+Mom3[3]^2)
                        if i>1
                            if TOF<LCIO.getTimeCont(hit,i-1)
                                count+=1
                            end
                        end
                        if TOF>10 || mom>10 || mom2>10 || costheta < costhetacut
                            continue
                        end
                        if !(haskey(cellinfo, cell))
                            push!(cellinfo, cell => (mcparticle, recoparticle, TOF))
                        else
                            if cellinfo[cell][3]>TOF
                                cellinfo[cell]=(mcparticle, recoparticle, TOF)
                            end
                        end
                        
                    end
                end
                infodict = Dict()
                for (key, value) in cellinfo
                    if !haskey(infodict, value[1]) 
                        push!(infodict, value[1] => (value[2],value[3]))
                    else
                        if infodict[value[1]][2]>value[3]
                             infodict[value[1]] = (value[2],value[3])
                        end
                    end
                end
                   for (key, value) in infodict
                        if getPDG(key)==211
                            push!(Momsspion, norm(getMomentum(value[1])))
                            push!(Momsspionmc, norm(getMomentum(key)))
                            push!(TOFsspion, value[2])
                        end
                        if getPDG(key)==321
                            push!(Momsskaon, norm(getMomentum(value[1])))
                            push!(Momsskaonmc, norm(getMomentum(key)))
                            push!(TOFsskaon, value[2])
                        end
                    end
                
            end
            if numEvents % 100 == 0
                println("processed ", numEvents, " events")
            end
        end
        println("read  $(filenum) files")
        
    end
    println("count is", count)
end

@time main()   
@save ("Momsskaon") Momsskaon
@save ("Momsspion") Momsspion
@save ("TOFsspion") TOFsspion
@save ("TOFsskaon") TOFsskaon

In [ ]:
scatter(Momsspion, TOFsspion, title = "Time of Flight (0-10 ns) vs Momentum (from PFOs)", 
    xlabel = "Momentum (GeV)", ylabel = "Time of Flight (ns)", 
    label = "pions", ms = 1.0, markerstrokewidth = 0 )
scatter!(Momsskaon, TOFsskaon, label = "kaons", ms = 0.1, markerstrokewidth=0)

savefig("kaons_vs_pions_PFOss")